In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
USstock_daily = pd.read_parquet('/quant_data/schedule_store/data/daily_US_Stock/crsp_dsf_20240620.pq')

In [3]:
USstock_daily.shape

(25695492, 31)

In [5]:
USstock_daily.ticker.nunique()

13592

In [5]:
USstock_daily[USstock_daily['ticker']=='AMZN']

,cusip,date,shrcd,shrcls,hexcd,trdstat,comnam,ticker,siccd,naics,...,dlprc,cfacpr,cfacshr,vwretd,vwretx,ewretd,ewretx,sprtrn,mcap,mcap_lag1
1149376,02313510,20000103,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,NaN,...,NaN,20.0,20.0,-0.006803,-0.006810,0.002878,0.002860,-0.009549,3.045784e+10,NaN
1149377,02313510,20000104,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,NaN,...,NaN,20.0,20.0,-0.039652,-0.039679,-0.017465,-0.017486,-0.038345,2.792323e+10,3.045784e+10
1149378,02313510,20000105,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,NaN,...,NaN,20.0,20.0,-0.000935,-0.001009,0.007821,0.007743,0.001922,2.376989e+10,2.792323e+10
1149379,02313510,20000106,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,NaN,...,NaN,20.0,20.0,-0.007391,-0.007547,0.004504,0.004453,0.000956,2.234285e+10,2.376989e+10
1149380,02313510,20000107,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,NaN,...,NaN,20.0,20.0,0.032516,0.032514,0.017008,0.016991,0.027090,2.370600e+10,2.234285e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155158,02313510,20221223,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,454110.0,...,NaN,1.0,1.0,0.005466,0.005395,0.002459,0.002195,0.005868,8.696910e+11,8.547966e+11
1155159,02313510,20221227,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,454110.0,...,NaN,1.0,1.0,-0.003928,-0.003997,-0.008489,-0.008633,-0.004050,8.471453e+11,8.696910e+11
1155160,02313510,20221228,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,454110.0,...,NaN,1.0,1.0,-0.012366,-0.012383,-0.010297,-0.010560,-0.012021,8.346993e+11,8.471453e+11
1155161,02313510,20221229,11.0,None,3,A,AMAZON COM INC,AMZN,7370.0,454110.0,...,NaN,1.0,1.0,0.018337,0.018135,0.022834,0.022336,0.017461,8.587752e+11,8.346993e+11


# Filter stocks

In [6]:
valid_stock = USstock_daily[['ticker','date']].groupby('ticker').agg(max_date=('date','max') ,min_date=('date','min'), duration=('date',lambda x: int(max(x))-int(min(x))))

In [7]:
valid_stock['max_date'] = pd.to_numeric(valid_stock['max_date'])
valid_stock['min_date'] = pd.to_numeric(valid_stock['min_date'])

In [8]:
valid_stock

,max_date,min_date,duration
ticker,,,
A,20221230,20000103,221127
AA,20221230,20000103,221127
AAAB,20030131,20010813,19318
AAABB,20010810,20000103,10707
AABC,20060104,20000103,60001
...,...,...,...
ZY,20221019,20210422,10597
ZYME,20221230,20221013,217
ZYNE,20221230,20150805,70425


In [9]:
filtered_stocks = valid_stock[(valid_stock['max_date']>20221229) & (valid_stock['min_date']<20200000)]

In [10]:
filtered_stocks.index.nunique()

3105

In [11]:

USstock_daily=USstock_daily.dropna(subset='ticker')

In [12]:
USstock_daily=USstock_daily.dropna(subset='ticker')

In [13]:
USstock_daily = USstock_daily[USstock_daily['ticker'].isin(filtered_stocks.index)]

In [14]:
USstock_daily['ticker'].nunique()

3105

In [15]:
ret = USstock_daily[['ticker', 'ret']]
mcap = USstock_daily[['ticker', 'mcap']]

ret = ret.dropna()
mcap = mcap.dropna()

In [16]:
# filter out stocks with insufficient return data
ret_2015 = ret[ret.index > '2015-01-01']
ret_mcount = ret_2015.groupby('ticker').resample('M').count()
insuf = ret_mcount[ret_mcount['ret']<10]['ret']
insuf=insuf.reset_index()
insuf_count = insuf.groupby('ticker').count()
insufficient_ret_data = insuf_count[insuf_count['ret']>50].index

/tmp/ipykernel_2631924/1079952424.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ret_mcount = ret_2015.groupby('ticker').resample('M').count()


In [17]:
USstock_daily = USstock_daily[~USstock_daily['ticker'].isin(insufficient_ret_data)]

# Resample date

In [18]:
ret = USstock_daily[['ticker', 'ret']]
mcap = USstock_daily[['ticker', 'mcap']]

ret = ret.dropna()
mcap = mcap.dropna()

In [19]:
# get geometric return
ret['ret'] = 1+ret['ret']
ret_monthly = ret.groupby('ticker').resample('M').prod()
ret_monthly = ret_monthly.reset_index()

# get mcap at the end of last month
mcap_monthly = mcap.groupby('ticker').resample('M').last()
mcap_monthly = mcap_monthly['mcap'].reset_index()

monthly_merge = pd.merge(ret_monthly, mcap_monthly, left_on=['ticker','date'],right_on=['ticker','date'])

/tmp/ipykernel_2631924/55760967.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ret_monthly = ret.groupby('ticker').resample('M').prod()
/tmp/ipykernel_2631924/55760967.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mcap_monthly = mcap.groupby('ticker').resample('M').last()


In [20]:
ret_quarterly = ret.groupby('ticker').resample('Q').prod()
ret_quarterly = ret_quarterly.reset_index()

# get mcap at the end of last month
mcap_quarterly = mcap.groupby('ticker').resample('Q').last()
mcap_quarterly = mcap_quarterly['mcap'].reset_index()

quarterly_merge = pd.merge(ret_quarterly, mcap_quarterly, left_on=['ticker','date'],right_on=['ticker','date'])

/tmp/ipykernel_2631924/2075495500.py:1: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  ret_quarterly = ret.groupby('ticker').resample('Q').prod()
/tmp/ipykernel_2631924/2075495500.py:5: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  mcap_quarterly = mcap.groupby('ticker').resample('Q').last()


In [21]:
ret_weekly = ret.groupby('ticker').resample('W').prod()
ret_weekly = ret_weekly.reset_index()

# get mcap at the end of last month
mcap_weekly = mcap.groupby('ticker').resample('W').last()
mcap_weekly = mcap_weekly['mcap'].reset_index()

weekly_merge = pd.merge(ret_weekly, mcap_weekly, left_on=['ticker','date'],right_on=['ticker','date'])

In [22]:
monthly_merge.to_csv('stock_monthly.csv',index=False)
quarterly_merge.to_csv('stock_quarterly.csv',index=False)
weekly_merge.to_csv('stock_weekly.csv',index=False)

# Rolling ret

In [5]:
stock_monthly = pd.read_csv('/home/group3/group3/Finalized/data/stock_monthly.csv')

In [10]:
stock_monthly[stock_monthly['ret']>10]

,ticker,date,ret,mcap
239627,GME,2021-01-31,17.250528,2.266778e+10
311917,KOSS,2021-01-31,18.604651,4.874240e+08
431899,PHUN,2019-01-31,20.883587,8.115943e+09
542297,TKAT,2021-03-31,10.897430,3.832140e+08
567199,UONE,2020-06-30,15.458013,3.203550e+07
579861,VNDA,2009-05-31,14.495054,3.901999e+08


In [13]:
USstock_daily[USstock_daily['ticker']=='PHUN'][['date', 'ret', 'openprc']]

,date,ret,openprc
18284829,20181228,0.068507,11.7500
18284830,20181231,0.315498,11.4036
18284831,20190102,NaN,NaN
18284832,20190103,0.870687,15.0000
18284833,20190104,1.026541,32.1000
...,...,...,...
18285834,20221223,0.007869,0.7700
18285835,20221227,0.000256,0.7800
18285836,20221228,-0.025592,0.7770
18285837,20221229,0.031254,0.7765


In [7]:
stock_monthly['ret'].sort_values()

490166     0.000261
197896     0.010088
16054      0.024420
532822     0.031708
268792     0.032258
            ...    
579861    14.495054
567199    15.458013
239627    17.250528
311917    18.604651
431899    20.883587
Name: ret, Length: 608628, dtype: float64

In [13]:
rolling_periods = [1,3,6,9,12,18,24]
rolling_ret = pd.DataFrame()
for i in rolling_periods:
    rolling_df = stock_monthly.groupby('ticker').apply(lambda x: x['ret'].rolling(window=i).apply(lambda x: x.prod())).reset_index()
    rolling_df['ret'] = rolling_df['ret'].shift(-i)
    rolling_df.rename(columns={'ret': f"{i}M_ret"},inplace=True)
    rolling_ret = pd.concat([rolling_ret,rolling_df[[f"{i}M_ret"]]],axis=1)

/tmp/ipykernel_3017298/94063445.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rolling_df = stock_monthly.groupby('ticker').apply(lambda x: x['ret'].rolling(window=i).apply(lambda x: x.prod())).reset_index()
/tmp/ipykernel_3017298/94063445.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rolling_df = stock_monthly.groupby('ticker').apply(lambda x: x['ret'].rolling(window=i).apply(lambda x: x.prod())

In [6]:
b = stock_monthly.groupby('ticker').apply(lambda x: x['ret'].rolling(window=12).apply(lambda x: x.prod()))

/tmp/ipykernel_3017298/2541645743.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  b = stock_monthly.groupby('ticker').apply(lambda x: x['ret'].rolling(window=12).apply(lambda x: x.prod()))


In [16]:
rolling_ret_df = pd.concat([stock_monthly[['ticker','date','mcap','ret']],rolling_ret],axis=1)

In [19]:
rolling_ret_df.to_csv('rolling_future_ret.csv',index=False)

# Get ret dataset for backtesting

In [85]:
USstock_daily['date'] = pd.to_datetime(USstock_daily['date'],format='%Y%m%d')
USstock_daily.set_index('date',inplace=True)

In [86]:
USstock_daily['ret'] = 1 + USstock_daily['ret']

In [87]:
ret = USstock_daily[['ticker','ret']].groupby(['ticker']).resample('M').prod()

/tmp/ipykernel_3095752/1691554350.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ret = USstock_daily[['ticker','ret']].groupby(['ticker']).resample('M').prod()


In [89]:
ret.unstack()

ret                                                         \
date   2000-01-31 2000-02-29 2000-03-31 2000-04-30 2000-05-31 2000-06-30   
ticker                                                                     
A        0.856104   1.569405   1.001205   0.852164   0.830747   1.001697   
AA       0.839610   0.986256   1.025546   0.923488   0.904311   0.992513   
AAAB          NaN        NaN        NaN        NaN        NaN        NaN   
AAABB    0.926471   0.920634   0.991379   0.852174   0.979592   1.166666   
AABC     0.970473   0.855983   0.900474   0.947369   1.116667   1.014926   
...           ...        ...        ...        ...        ...        ...   
ZY            NaN        NaN        NaN        NaN        NaN        NaN   
ZYME          NaN        NaN        NaN        NaN        NaN        NaN   
ZYNE          NaN        NaN        NaN        NaN        NaN        NaN   
ZYXI          NaN        NaN        NaN        NaN        NaN        NaN   
ZZ            NaN        NaN        NaN        NaN        NaN        NaN   

                                                    ...                        \
date   2000-07-31 2000-08-31 2000-09-30 2000-10-31  ... 2022-03-31 2022-04-30   
ticker                                              ...                         
A        0.552542   1.480060   0.811399   0.946360  ...   1.015113   0.902713   
AA       1.043103   1.103516   0.761278   1.133335  ...   1.196382   0.753081   
AAAB          NaN        NaN        NaN        NaN  ...        NaN        NaN   
AAABB    1.178574   1.113637   0.952379   0.907142  ...        NaN        NaN   
AABC     0.975489   0.984924   1.030612   1.009901  ...        NaN        NaN   
...           ...        ...        ...        ...  ...        ...        ...   
ZY            NaN        NaN        NaN        NaN  ...   0.754569   0.581313   
ZYME          NaN        NaN        NaN        NaN  ...        NaN        NaN   
ZYNE          NaN        NaN        NaN        NaN  ...   0.907077   0.687805   
ZYXI          NaN        NaN        NaN        NaN  ...   0.990462   1.022473   
ZZ            NaN        NaN        NaN        NaN  ...        NaN        NaN   

                                                                          \
date   2022-05-31 2022-06-30 2022-07-31 2022-08-31 2022-09-30 2022-10-31   
ticker                                                                     
A        1.069509   0.931092   1.131062   0.956377   0.947758   1.140106   
AA       0.911834   0.738497   1.116498   0.974290   0.680275   1.162506   
AAAB          NaN        NaN        NaN        NaN        NaN        NaN   
AAABB         NaN        NaN        NaN        NaN        NaN        NaN   
AABC          NaN        NaN        NaN        NaN        NaN        NaN   
...           ...        ...        ...        ...        ...        ...   
ZY       0.845237   0.866196   2.008127   0.959515   1.172997   0.792377   
ZYME          NaN        NaN        NaN        NaN        NaN   1.468378   
ZYNE     0.679078   1.190599   0.999999   1.026314   0.625639   0.997405   
ZYXI     1.144428   1.094650   1.075186   1.041958   1.014541   1.256893   
ZZ            NaN        NaN        NaN        NaN        NaN        NaN   

                              
date   2022-11-30 2022-12-31  
ticker                        
A        1.120205   0.967059  
AA       1.284397   0.907041  
AAAB          NaN        NaN  
AAABB         NaN        NaN  
AABC          NaN        NaN  
...           ...        ...  
ZY            NaN        NaN  
ZYME     1.039030   1.018136  
ZYNE     0.735926   0.986415  
ZYXI     1.203510   1.013849  
ZZ            NaN        NaN  

[13592 rows x 276 columns]

In [90]:
ret2 = ret.reset_index().groupby('ticker').shift(-1)['ret']
ret['ret'] = ret2.values

In [92]:
ret.unstack()

ret                                                         \
date   2000-01-31 2000-02-29 2000-03-31 2000-04-30 2000-05-31 2000-06-30   
ticker                                                                     
A        1.569405   1.001205   0.852164   0.830747   1.001697   0.552542   
AA       0.986256   1.025546   0.923488   0.904311   0.992513   1.043103   
AAAB          NaN        NaN        NaN        NaN        NaN        NaN   
AAABB    0.920634   0.991379   0.852174   0.979592   1.166666   1.178574   
AABC     0.855983   0.900474   0.947369   1.116667   1.014926   0.975489   
...           ...        ...        ...        ...        ...        ...   
ZY            NaN        NaN        NaN        NaN        NaN        NaN   
ZYME          NaN        NaN        NaN        NaN        NaN        NaN   
ZYNE          NaN        NaN        NaN        NaN        NaN        NaN   
ZYXI          NaN        NaN        NaN        NaN        NaN        NaN   
ZZ            NaN        NaN        NaN        NaN        NaN        NaN   

                                                    ...                        \
date   2000-07-31 2000-08-31 2000-09-30 2000-10-31  ... 2022-03-31 2022-04-30   
ticker                                              ...                         
A        1.480060   0.811399   0.946360   1.126853  ...   0.902713   1.069509   
AA       1.103516   0.761278   1.133335   0.986797  ...   0.753081   0.911834   
AAAB          NaN        NaN        NaN        NaN  ...        NaN        NaN   
AAABB    1.113637   0.952379   0.907142   1.102363  ...        NaN        NaN   
AABC     0.984924   1.030612   1.009901   0.960784  ...        NaN        NaN   
...           ...        ...        ...        ...  ...        ...        ...   
ZY            NaN        NaN        NaN        NaN  ...   0.581313   0.845237   
ZYME          NaN        NaN        NaN        NaN  ...        NaN        NaN   
ZYNE          NaN        NaN        NaN        NaN  ...   0.687805   0.679078   
ZYXI          NaN        NaN        NaN        NaN  ...   1.022473   1.144428   
ZZ            NaN        NaN        NaN        NaN  ...        NaN        NaN   

                                                                          \
date   2022-05-31 2022-06-30 2022-07-31 2022-08-31 2022-09-30 2022-10-31   
ticker                                                                     
A        0.931092   1.131062   0.956377   0.947758   1.140106   1.120205   
AA       0.738497   1.116498   0.974290   0.680275   1.162506   1.284397   
AAAB          NaN        NaN        NaN        NaN        NaN        NaN   
AAABB         NaN        NaN        NaN        NaN        NaN        NaN   
AABC          NaN        NaN        NaN        NaN        NaN        NaN   
...           ...        ...        ...        ...        ...        ...   
ZY       0.866196   2.008127   0.959515   1.172997   0.792377        NaN   
ZYME          NaN        NaN        NaN        NaN        NaN   1.039030   
ZYNE     1.190599   0.999999   1.026314   0.625639   0.997405   0.735926   
ZYXI     1.094650   1.075186   1.041958   1.014541   1.256893   1.203510   
ZZ            NaN        NaN        NaN        NaN        NaN        NaN   

                              
date   2022-11-30 2022-12-31  
ticker                        
A        0.967059        NaN  
AA       0.907041        NaN  
AAAB          NaN        NaN  
AAABB         NaN        NaN  
AABC          NaN        NaN  
...           ...        ...  
ZY            NaN        NaN  
ZYME     1.018136        NaN  
ZYNE     0.986415        NaN  
ZYXI     1.013849        NaN  
ZZ            NaN        NaN  

[13592 rows x 276 columns]

In [73]:
unstacked = ret.unstack()['ret']

In [97]:
df = unstacked.loc[:,:'2022-12-30'].T

In [98]:
df.to_csv('universe_ret.csv')

In [3]:
ret = pd.read_csv('/home/group3/group3/Finalized/data/universe_ret.csv',index_col=0)


In [4]:
ret-=1

In [5]:
ret.to_csv('universe_ret.csv')

In [15]:
USstock_daily[USstock_daily['ticker']=='IKT']

,cusip,date,shrcd,shrcls,hexcd,trdstat,comnam,ticker,siccd,naics,...,dlprc,cfacpr,cfacshr,vwretd,vwretx,ewretd,ewretx,sprtrn,mcap,mcap_lag1
12005339,45719W10,20201223,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,0.001445,0.001420,0.010842,0.010702,0.000746,99495000.00,NaN
12005340,45719W10,20201224,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,0.002330,0.002195,-0.000277,-0.000458,0.003537,98490000.00,99495000.00
12005341,45719W10,20201228,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,0.003847,0.003836,0.001350,0.001304,0.008723,93867000.00,98490000.00
12005342,45719W10,20201229,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,-0.003652,-0.003660,-0.008483,-0.008656,-0.002227,76380000.00,93867000.00
12005343,45719W10,20201230,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,0.003291,0.003124,0.010155,0.009754,0.001342,71958000.00,76380000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12005843,45719W10,20221223,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,0.005466,0.005395,0.002459,0.002195,0.005868,12486103.65,12108960.00
12005844,45719W10,20221227,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,-0.003928,-0.003997,-0.008489,-0.008633,-0.004050,11604420.00,12486103.65
12005845,45719W10,20221228,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,-0.012366,-0.012383,-0.010297,-0.010560,-0.012021,12363752.70,11604420.00
12005846,45719W10,20221229,11.0,None,3,A,INHIBIKASE THERAPEUTICS INC,IKT,9999.0,325412.0,...,NaN,1.0,1.0,0.018337,0.018135,0.022834,0.022336,0.017461,12860724.60,12363752.70
